# Vacation Py
## Using API to make maps of cities from WeatherPy and find Hotels

In [3]:
# Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Make DataFrame from list of cities from WeatherPy

In [4]:
#pull cities from the csv file from WeatherPy and put into a DataFrame
city_data = pd.read_csv("../Output_Data/Cities.csv")
city_data = city_data.drop("Unnamed: 0", axis=1)
city_data.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Paramonga,-10.67,-77.83,66.65,87,99,7.45,PE,1607580098
1,Olafsvik,64.89,-23.71,35.87,85,100,23.87,IS,1607580098
2,Provideniya,64.38,-173.30,10.40,66,40,6.71,RU,1607580098
3,Qasigiannguit,68.82,-51.19,17.60,44,60,2.24,GL,1607580098
4,George Town,5.41,100.34,91.40,70,20,8.05,MY,1607580098


In [5]:
#define variables for map
cities = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"].astype(float)

### Humidity Heatmap

In [6]:
#map dataframe with heatmap using the humidity
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights = humidity, dissipating=False, max_intensity =100, point_radius=2)
figure_layout = {'width': '100px', 'margin': '0 auto 0 auto'}
gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#which cities have "ideal weather"
city_data[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"] < 80) & 
                          (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Hermanus,-34.42,19.23,71.01,75,0,2.84,ZA,1607580102
141,Doha,25.29,51.53,71.60,56,0,8.05,QA,1607580122
153,Uthal,25.81,66.62,78.93,35,0,6.11,PK,1607580120
180,Minas,-34.38,-55.24,71.01,77,0,5.82,UY,1607579967
259,Belmonte,-15.86,-38.88,73.40,88,0,4.70,BR,1607580143
322,Pilar,-9.60,-35.96,71.60,94,0,3.36,BR,1607580115
414,Maceió,-9.67,-35.74,71.60,94,0,3.36,BR,1607580168
443,Barentu,15.11,37.59,75.34,37,0,9.04,ER,1607580173
525,Alenquer,-1.94,-54.74,78.80,88,0,9.17,BR,1607580186


## Ideal Weather DataFrame

In [8]:
#take those results and turn into a dataframe
ideal_weather = city_data.loc[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"] < 80) & 
                           (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]
ideal_weather

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Hermanus,-34.42,19.23,71.01,75,0,2.84,ZA,1607580102
141,Doha,25.29,51.53,71.60,56,0,8.05,QA,1607580122
153,Uthal,25.81,66.62,78.93,35,0,6.11,PK,1607580120
180,Minas,-34.38,-55.24,71.01,77,0,5.82,UY,1607579967
259,Belmonte,-15.86,-38.88,73.40,88,0,4.70,BR,1607580143
322,Pilar,-9.60,-35.96,71.60,94,0,3.36,BR,1607580115
414,Maceió,-9.67,-35.74,71.60,94,0,3.36,BR,1607580168
443,Barentu,15.11,37.59,75.34,37,0,9.04,ER,1607580173
525,Alenquer,-1.94,-54.74,78.80,88,0,9.17,BR,1607580186


## Finding Hotels for cities with Ideal Weather

In [9]:
# #adding column to data set
# ideal_weather["Hotel Name"] = ""

In [10]:
#API Reuqest for hotel information
#variables for url
radius = "5000"
place_type = "lodging"

#for loop for the cities in the ideal weather dataframe
for index, city in ideal_weather.iterrows():
    #variables for url
    latitude = city["Latitude"]
    longitude = city["Longitude"]
    location = f"{latitude},{longitude}"
    #base url
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={place_type}&key={g_key}"
    response = requests.get(url).json()
    
    #use the try function in case hotel not found
    try:
        #store hotel name into dataframe
        hotel_name = response["results"][0]["name"]
        ideal_weather.loc[index, "Hotel Name"] = hotel_name
    #do not need print out, just need to skip    
    except IndexError:
        continue
#print updated dataframe        
ideal_weather.head()

C:\Users\Shadow\anaconda3\envs\PythonAdv\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Shadow\anaconda3\envs\PythonAdv\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Hermanus,-34.42,19.23,71.01,75,0,2.84,ZA,1607580102,Misty Waves Boutique Hotel
141,Doha,25.29,51.53,71.60,56,0,8.05,QA,1607580122,Doha Marriott Hotel
153,Uthal,25.81,66.62,78.93,35,0,6.11,PK,1607580120,Al Madina Hotal
180,Minas,-34.38,-55.24,71.01,77,0,5.82,UY,1607579967,Hotel Minas
259,Belmonte,-15.86,-38.88,73.40,88,0,4.70,BR,1607580143,Pousada Monte Carmelo O Pão


## Hotel DataFrame

In [10]:
#making a new dataframe with just info for hotel
hotel_df = ideal_weather[["Hotel Name","City","Country","Latitude","Longitude"]]
hotel_df = hotel_df.rename(columns = {"Latitude":"Lat", "Longitude":"Lng"})
hotel_df

,Hotel Name,City,Country,Lat,Lng
23,Misty Waves Boutique Hotel,Hermanus,ZA,-34.42,19.23
141,Doha Marriott Hotel,Doha,QA,25.29,51.53
153,Al Madina Hotal,Uthal,PK,25.81,66.62
180,Hotel Minas,Minas,UY,-34.38,-55.24
259,Pousada Monte Carmelo O Pão,Belmonte,BR,-15.86,-38.88
322,Pousada e churrascaria Manguaba,Pilar,BR,-9.60,-35.96
414,Mercure Maceio Pajucara Hotel,Maceió,BR,-9.67,-35.74
443,,Barentu,ER,15.11,37.59
525,Josilene Castro,Alenquer,BR,-1.94,-54.74


## Hotel Map

In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
#mappling the previous map again but now with the hotel information for some of the cities
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(cities, weights = humidity, dissipating=False, max_intensity =100, point_radius=2)
fig.add_layer(heat_layer)
figure_layout = {'width': '100px', 'margin': '0 auto 0 auto'}
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

## Analysis

From the list of cites, for the current date, there are only 8 cities that have "ideal weather" right now. 